# 基于视频分类的打架识别

## 项目说明

2020年全球安防镜头出货量达3.75亿。随着安防摄像头覆盖范围越来越广，通过人工监视镜头画面识别异常行为变得不太现实。AI+安防，智慧城市解决方案受到了越来越多的关注。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/941c5da5ddf742f7bfcbc6becf88ad9b011bd8b398ff40699602398fa46dc188" width=700 ></img></center>

<center>图1 全球安防镜头出货量</center>

本项目基于PaddleVideo视频开发套件训练打架识别模型，然后将训练好的模型集成到PaddleDetection的PP-Human中，助力行人行为分析。

项目难点：
* **行为的动态性**：相比于基于静态图像就能判断的行为，如打球、打电话等，打架对时序有着较强的依赖。利用目标检测技术去识别打架行为容易出现误检或者漏检的情况。因此需要引入时序信息来判定是否有打架行为发生。
* **人员的密集性**：监控画面中的人的数目存在多变的情况，如火车站、超市等区域人流量较大，基于骨骼点进行行为识别对骨骼点提取的精准度有着较强的依赖。所以如果人员较密集，则基于骨骼点技术识别打架行为存在着局限性。
* **环境的复杂性**：监控场景多样，如白天、晚上，室内、室外，多种拍摄角度拍摄距离等，这些都对模型的泛化性能提出了更高的要求。

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/e694aae20f2646f48787268d9308fd93dc53a6b4fbe74cefab6fe795a44e4a8c" width=700 ></img></center>

<center>图2 密集人群骨骼点提取局限性（注：图片来自UBI FIGHTS数据集）</center>


**如需更多技术交流与合作欢迎扫码加入官方交流群：**

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/75fe8c650e5b42bf8bc0a4d5ae9ce12aa8a8030dcd154d21819d2b486e52649a" width="25%" height="25%"> <br /><center/>







## 安装说明
本项目依赖PaddleVideo和PaddleDetection两个开发套件，下面一一进行安装。

### PaddleVideo安装

下载PaddleVideo源码：

In [ ]:
# 速度可能较慢
!git clone https://github.com/PaddlePaddle/PaddleVideo.git

或直接解压本项目提供的源码：

In [1]:
%cd /home/aistudio
!unzip PaddleVideo-develop.zip
!mv PaddleVideo-develop PaddleVideo


  inflating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/reader/reader_utils.py  
  inflating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/reader/tsminf_reader.py  
   creating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/utils/
 extracting: PaddleVideo-develop/applications/FootballAction/predict/action_detect/utils/__init__.py  
  inflating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/utils/config_utils.py  
  inflating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/utils/preprocess.py  
  inflating: PaddleVideo-develop/applications/FootballAction/predict/action_detect/utils/process_result.py  
   creating: PaddleVideo-develop/applications/FootballAction/predict/configs/
  inflating: PaddleVideo-develop/applications/FootballAction/predict/configs/configs.yaml  
  inflating: PaddleVideo-develop/applications/FootballAction/predict/configs/index_label_football_8

安装依赖库：

In [2]:
%cd /home/aistudio/PaddleVideo
!pip install --upgrade -r requirements.txt --user

/home/aistudio/PaddleVideo
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 36.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 37.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 12.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 22.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 40.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### PaddleDetection安装

本项目提供了用到的PaddleDetection代码，直接解压即可使用：

In [3]:
%cd /home/aistudio
!unzip /home/aistudio/data/data156736/PaddleDetection-develop.zip -d /home/aistudio/

  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_fpn_reader.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_cbr101_vd_dual_fpn_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_cbr50_vd_dual_fpn_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r101_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r101_fpn_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r101_fpn_2x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r101_vd_fpn_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r101_vd_fpn_2x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r34_fpn_1x.yml  
  inflating: /home/aistudio/PaddleDetection-develop/static/configs/faster_rcnn_r34_vd_fpn_1x.yml  
  inflating: /

安装环境：

In [4]:
%cd /home/aistudio/PaddleDetection-develop/
!pip install -r requirements.txt
# 编译安装paddledet
!python setup.py install

copying ppdet/core/workspace.py -> build/lib/ppdet/core
copying ppdet/core/__init__.py -> build/lib/ppdet/core
creating build/lib/ppdet/modeling
copying ppdet/modeling/layers.py -> build/lib/ppdet/modeling
copying ppdet/modeling/keypoint_utils.py -> build/lib/ppdet/modeling
copying ppdet/modeling/bbox_utils.py -> build/lib/ppdet/modeling
copying ppdet/modeling/ops.py -> build/lib/ppdet/modeling
copying ppdet/modeling/cls_utils.py -> build/lib/ppdet/modeling
copying ppdet/modeling/__init__.py -> build/lib/ppdet/modeling
copying ppdet/modeling/shape_spec.py -> build/lib/ppdet/modeling
copying ppdet/modeling/initializer.py -> build/lib/ppdet/modeling
copying ppdet/modeling/post_process.py -> build/lib/ppdet/modeling
creating build/lib/ppdet/slim
copying ppdet/slim/ofa.py -> build/lib/ppdet/slim
copying ppdet/slim/unstructured_prune.py -> build/lib/ppdet/slim
copying ppdet/slim/distill.py -> build/lib/ppdet/slim
copying ppdet/slim/prune.py -> build/lib/ppdet/slim
copying ppdet/slim/quant.p

## 数据准备
本项目为大家提供了5个公开数据集合并后的数据。5个公开数据集具体如下：

<center><img src="https://ai-studio-static-online.cdn.bcebos.com/ef93638e760742308ffd596414cca44ff2b8930986b4440ba3d6bff1255a5a3a" width=700 ></img></center>

<center>图3 5个公开数据集详情</center>

总共包含打架视频1980个，非打架视频1871个，共3851个视频。数据下载：https://aistudio.baidu.com/aistudio/datasetdetail/149085 。

由于数据量较大，下面的实验仅基于少量数据演示。数据的存放目录为：work/dataset。

为了加快训练速度，将视频进行抽帧。此处借助PaddleVideo套件实现。

In [5]:
%cd /home/aistudio/PaddleVideo
!python data/ucf101/extract_rawframes.py /home/aistudio/work/dataset/ \
   /home/aistudio/work/rawframes/ --level 2 --ext mp4

/home/aistudio/PaddleVideo
Creating folder: /home/aistudio/work/rawframes/
Creating folder: /home/aistudio/work/rawframes/fight
Creating folder: /home/aistudio/work/rawframes/nofight
Reading videos from folder:  /home/aistudio/work/dataset/
Extension of videos:  mp4
Total number of videos found:  5
fight/V_998 done with 150 frames
fight/V_999 done with 150 frames
nofight/NV_998 done with 154 frames
nofight/NV_997 done with 152 frames
nofight/NV_999 done with 168 frames


接下来将视频按照0.8:0.2的比例划分成训练集和测试集：

In [6]:
%cd /home/aistudio

/home/aistudio


In [7]:
import os
import glob
import random
import fnmatch
import re

class_id = {
    "nofight":0,
    "fight":1
}

def get_list(path,key_func=lambda x: x[-11:], rgb_prefix='img_', level=1):
    if level == 1:
        frame_folders = glob.glob(os.path.join(path, '*'))
    elif level == 2:
        frame_folders = glob.glob(os.path.join(path, '*', '*'))
    else:
        raise ValueError('level can be only 1 or 2')

    def count_files(directory):
        lst = os.listdir(directory)
        cnt = len(fnmatch.filter(lst, rgb_prefix + '*')) 
        return cnt

    # check RGB
    video_dict = {}
    for f in frame_folders:
        cnt = count_files(f)
        k = key_func(f)
        if level==2:
            k = k.split("/")[0]

        video_dict[f]=str(cnt)+" "+str(class_id[k])
    
    return video_dict

def fight_splits(video_dict, train_percent=0.8):
    videos = list(video_dict.keys())

    train_num = int(len(videos)*train_percent)

    train_list = []
    val_list = []

    random.shuffle(videos)

    for i in range(train_num):
        train_list.append(videos[i]+" "+str(video_dict[videos[i]]))
    for i in range(train_num,len(videos)):
        val_list.append(videos[i]+" "+str(video_dict[videos[i]]))

    print("train:",len(train_list),",val:",len(val_list))
    
    with open("fight_train_list.txt","w") as f:
        for item in train_list:
            f.write(item+"\n")
    
    with open("fight_val_list.txt","w") as f:
        for item in val_list:
            f.write(item+"\n")

frame_dir = "/home/aistudio/work/rawframes"
level = 2
train_percent = 0.8 

if level == 2:
    def key_func(x):
        return '/'.join(x.split('/')[-2:])
else:
    def key_func(x):
        return x.split('/')[-1]

video_dict = get_list(frame_dir, key_func=key_func, level=level)    
print("number:",len(video_dict))

fight_splits(video_dict, train_percent)


number: 5
train: 4 ,val: 1


生成fight_train_list.txt和fight_val_list.txt两个文件。

## 模型选择
行为识别常用的技术有检测技术、骨骼点技术、时空动作检测技术等，下面简单介绍了这三种技术在打架识别任务中的局限性：

* **检测技术**：缺少时序信息，容易出现误检或漏检的情况。
* **骨骼点技术**：人员密集骨骼点提取效果不好，对最终识别结果造成影响。
* **时空动作检测技术**：速度慢、难度高，实际应用非必需。

基于视频分类的方法进行打架识别可以利用时序信息和场景信息，泛化能力相对较强，而且推理速度适当。因此，本项目采用基于视频分类的方法识别打架行为。

考虑到速度和精度，本项目采用具有飞桨特色的高精度2D实用视频分类模型PP-TSM模型进行视频二分类。

## 模型训练

下载预训练模型：


In [8]:
%cd /home/aistudio/
!wget https://videotag.bj.bcebos.com/PaddleVideo/PretrainModel/ResNet50_vd_ssld_v2_pretrained.pdparams

/home/aistudio
--2024-12-27 22:54:07--  https://videotag.bj.bcebos.com/PaddleVideo/PretrainModel/ResNet50_vd_ssld_v2_pretrained.pdparams
正在解析主机 videotag.bj.bcebos.com (videotag.bj.bcebos.com)... 182.61.200.229, 182.61.200.195, 2409:8c04:1001:1203:0:ff:b0bb:4f27
正在连接 videotag.bj.bcebos.com (videotag.bj.bcebos.com)|182.61.200.229|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 155155185 (148M) [application/octet-stream]
正在保存至: “ResNet50_vd_ssld_v2_pretrained.pdparams”

ResNet50_vd_ssld_v2 100%[===================>] 147.97M  21.6MB/s    in 11s     

2024-12-27 22:54:18 (13.6 MB/s) - 已保存 “ResNet50_vd_ssld_v2_pretrained.pdparams” [155155185/155155185])



In [9]:
# 准备相关依赖
!pip install av
!pip install decord

Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 9.2 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Looking in indexes: https://mirror.baidu.com/pypi/simple/, https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 15.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


模型训练：

In [11]:
# 单卡训练
%cd /home/aistudio/PaddleVideo/
!python main.py --validate -c /home/aistudio/pptsm_fight_frames_dense.yaml

/home/aistudio/PaddleVideo
No module named 'skimage', [scikit-image] package and it's dependencies is required for ADDS.
No module named 'SimpleITK', [SimpleITK] package and it's dependencies is required for PP-Care.
No module named 'SimpleITK', [SimpleITK] package and it's dependencies is required for PP-Care.
No module named 'lmdb', [lmdb] package and it's dependencies is required for ActBERT.
No module named 'lmdb', [lmdb] package and it's dependencies is required for ActBERT.
[12/27 23:04:51] DALI is not installed, you can improve performance if use DALI
[12/27 23:04:51] DATASET : 
[12/27 23:04:51]     batch_size : 1
[12/27 23:04:51]     num_workers : 0
[12/27 23:04:51]     test : 
[12/27 23:04:51]         data_prefix : 
[12/27 23:04:51]         file_path : /home/aistudio/fight_val_list.txt
[12/27 23:04:51]         format : FrameDataset
[12/27 23:04:51]         suffix : img_{:05}.jpg
[12/27 23:04:51]     test_batch_size : 1
[12/27 23:04:51]     train : 
[12/27 23:04:51]         dat

In [ ]:
# 多卡训练
!export CUDA_VISIBLE_DEVICES=0,1,2,3
!python -B -m paddle.distributed.launch --gpus=“0,1,2,3” \
   --log_dir=log_pptsm_dense  main.py  --validate \
   -c /home/aistudio/pptsm_fight_frames_dense.yaml

## 模型评估
本项目提供了训练好的模型，路径为：/home/aistudio/ppTSM_fight_VideoMix_target_size_320_best.pdparams。

In [12]:
%cd /home/aistudio/PaddleVideo/
!python main.py --test -c /home/aistudio/pptsm_fight_frames_dense.yaml \
   -w /home/aistudio/ppTSM_fight_best.pdparams

/home/aistudio/PaddleVideo
No module named 'skimage', [scikit-image] package and it's dependencies is required for ADDS.
No module named 'SimpleITK', [SimpleITK] package and it's dependencies is required for PP-Care.
No module named 'SimpleITK', [SimpleITK] package and it's dependencies is required for PP-Care.
No module named 'lmdb', [lmdb] package and it's dependencies is required for ActBERT.
No module named 'lmdb', [lmdb] package and it's dependencies is required for ActBERT.
[12/27 23:09:24] DALI is not installed, you can improve performance if use DALI
[12/27 23:09:24] DATASET : 
[12/27 23:09:24]     batch_size : 1
[12/27 23:09:24]     num_workers : 0
[12/27 23:09:24]     test : 
[12/27 23:09:24]         data_prefix : 
[12/27 23:09:24]         file_path : /home/aistudio/fight_val_list.txt
[12/27 23:09:24]         format : FrameDataset
[12/27 23:09:24]         suffix : img_{:05}.jpg
[12/27 23:09:24]     test_batch_size : 1
[12/27 23:09:24]     train : 
[12/27 23:09:24]         dat

## 模型推理

导出inference模型：

In [13]:
%cd /home/aistudio/PaddleVideo/
!python tools/export_model.py -c /home/aistudio/pptsm_fight_frames_dense.yaml \
                                -p /home/aistudio/ppTSM_fight_best.pdparams \
                                -o inference/ppTSM

/home/aistudio/PaddleVideo
Building model(ppTSM)...
W1227 23:10:05.087051  6073 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 11.2
W1227 23:10:05.092643  6073 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
Loading params from (/home/aistudio/ppTSM_fight_best.pdparams)...
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/inplace_utils.py:31: UserWarning: In static mode, reshape_() is the same as reshape() and does not perform inplace operation.
  format(func.__name__, origin_api_name))
model (ppTSM) has been already saved in (inference/ppTSM).


使用预测引擎推理:

In [14]:
%cd /home/aistudio/PaddleVideo/
!python tools/predict.py --input_file /home/aistudio/work/fight.mp4 \
                           --config /home/aistudio/pptsm_fight_frames_dense.yaml \
                           --model_file inference/ppTSM/ppTSM.pdmodel \
                           --params_file inference/ppTSM/ppTSM.pdiparams \
                           --use_gpu=True \
                           --use_tensorrt=False

/home/aistudio/PaddleVideo
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
No module named 'skimage', [scikit-image] package and it's dependencies is required for ADDS.
No module named 'SimpleITK', [SimpleIT

## 模型优化

### VideoMix
[VideoMix](https://arxiv.org/abs/2012.03457)是视频数据增强的方法之一，是对图像数据增强CutMix的扩展，可以缓解模型的过拟合问题。

与Mixup将两个视频片段的每个像素点按照一定比例融合不同的是，VideoMix是每个像素点要么属于片段A要么属于片段B。输出结果是两个片段原始标签的加权和，权重是两个片段各自的比例。

在配置文件中对应：
```
MIX:
    name: "VideoMix"
    cutmix_prob: 0.5
    mixup_alpha: 0.2
    cutmix_alpha: 1.0
```

本项目在baseline的基础上加入VideoMix数据增强后，精度由87.53%提升至88.01%。

### 更大的分辨率

由于监控摄像头角度、距离等问题，存在监控画面下人比较小的情况，小目标行为的识别较困难，本项目尝试增大输入图像的分辨率，具体修改如下：
```
transform: #Mandotary, image transfrom operator
            - Scale:
                short_size: 340 #256
            - MultiScaleCrop:
                target_size: 340 #256
            - RandomCrop:
                target_size: 320 #224
            - RandomFlip:
            - Image2Array:
            - Normalization:
                mean: [0.485, 0.456, 0.406]
                std: [0.229, 0.224, 0.225]
```

通过增大输入图像分辨率，模型精度由88.01%提升至89.06%。


## 基于PaddleDetection pipeline实现打架识别

将打架识别功能整合到PaddleDetection的PP-Human中，助力行人行为分析。

开启打架识别功能需修改配置文件deploy/pipeline/config/infer_cfg_pphuman.yml中VIDEO_ACTION的enable为**True**。

下载打架识别预测部署模型，并解压：

In [15]:
%cd /home/aistudio/PaddleDetection-develop/
!wget https://videotag.bj.bcebos.com/PaddleVideo-release2.3/ppTSM_fight.zip
!mkdir /home/aistudio/PaddleDetection-develop/output_inference
!unzip ppTSM_fight.zip -d /home/aistudio/PaddleDetection-develop/output_inference

/home/aistudio/PaddleDetection-develop
--2024-12-27 23:11:07--  https://videotag.bj.bcebos.com/PaddleVideo-release2.3/ppTSM_fight.zip
正在解析主机 videotag.bj.bcebos.com (videotag.bj.bcebos.com)... 182.61.200.195, 182.61.200.229, 2409:8c04:1001:1203:0:ff:b0bb:4f27
正在连接 videotag.bj.bcebos.com (videotag.bj.bcebos.com)|182.61.200.195|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 87616865 (84M) [application/zip]
正在保存至: “ppTSM_fight.zip”

ppTSM_fight.zip     100%[===================>]  83.56M  30.6MB/s    in 2.7s    

2024-12-27 23:11:09 (30.6 MB/s) - 已保存 “ppTSM_fight.zip” [87616865/87616865])

Archive:  ppTSM_fight.zip
   creating: /home/aistudio/PaddleDetection-develop/output_inference/ppTSM/
  inflating: /home/aistudio/PaddleDetection-develop/output_inference/ppTSM/model.pdiparams  
   creating: /home/aistudio/PaddleDetection-develop/output_inference/ppTSM/.ipynb_checkpoints/
  inflating: /home/aistudio/PaddleDetection-develop/output_inference/ppTSM/model.pdmodel  
  inflating: /home/aistudio/

运行下面的命令进行打架识别，参数说明如下：
* config：配置文件路径；
* video_file：视频路径；
* device：推理环境。

In [16]:
%cd /home/aistudio/PaddleDetection-develop/
# 打架识别，指定配置文件路径和测试视频
!python deploy/pipeline/pipeline.py --config deploy/pipeline/config/infer_cfg_pphuman.yml \
 --video_file=/home/aistudio/work/72.mp4 \
 --device=gpu 


/home/aistudio/PaddleDetection-develop
deploy/pipeline/pipeline.py:24: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
-----------  Running Arguments -----------
ATTR:
  basemode: idbased
  batch_size: 8
  enable: false
  model_dir: output_inference/PPLCNet_x1_0_person_attribute_945_infer/
DET:
  batch_size: 1
  model_dir: output_inference/mot_ppyoloe_l_36e_pipeline/
ID_BASED_CLSACTION:
  basemode: idbased
  batch_size: 8
  display_frames: 80
  enable: false
  model_dir: output_inference/PPHGNet_tiny_calling_halfbody
  skip_frame_num: 2
  threshold: 0.8
ID_BASED_DETACTION:
  basemode: idbased
  batch_size: 8
  display_frames: 80
  enable: false
  model_dir: output_inference/ppyoloe_crn_s_80e_smoking_visdrone
  skip_frame_num: 2
  threshold: 0.6
KPT:
  batch_size: 8
  model_dir: output_inference/dark_hrnet_w32_256x192/
MOT:
  basemode: idbas

输出结果在/home/aistudio/PaddleDetection-develop/output/文件夹中。